In [16]:
%pip install tensorflow==1.13.1 keras==2.3.0

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement tensorflow==1.13.1 (from versions: 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.1, 2.16.0rc0, 2.16.1)
ERROR: No matching distribution found for tensorflow==1.13.1

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [26]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sofarooq/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [24]:
import nltk
from nltk.corpus import movie_reviews
from gensim.models import Word2Vec
import string

import numpy as np
from gensim.models import Word2Vec
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [27]:
# Download NLTK data 
nltk.download('movie_reviews')

# Load the movie reviews from NLTK's IMDb dataset
reviews = [movie_reviews.words(fileid) for fileid in movie_reviews.fileids()]

# Preprocess the text
stop_words = set(nltk.corpus.stopwords.words('english'))
punctuation = set(string.punctuation)
filtered_reviews = []

for review in reviews:
    words = [word.lower() for word in review if word.lower() not in stop_words and word.lower() not in punctuation]
    filtered_reviews.append(words)

[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/sofarooq/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


In [28]:
# Train Word2Vec model
model = Word2Vec(filtered_reviews, vector_size=100, window=5, min_count=1, workers=4)

# Get word vector for a specific word (example)
word = "movie"
word_vector = model.wv[word]

print(f"Word Vector for '{word}':\n{word_vector}")

Word Vector for 'movie':
[-1.1424881   1.3501904   0.11758545  0.11456764 -0.87633425 -0.33911118
  0.9897299   1.1941618  -1.0107526  -0.89252156 -0.6463401  -1.3028039
  0.22384821  1.0236348   0.3913481  -0.20928949  0.3228618  -0.252683
  0.17742872 -2.6832743   1.9265242  -0.25758314  0.63546634  0.08225012
 -0.1506141   0.55729556 -0.7024286   0.72482336 -0.49098688 -0.1040457
  0.8487141   0.7597629  -0.09681227 -0.9932533  -1.1163546   0.72968787
  1.2967625  -0.6498055  -0.6669654  -0.651981    0.03503898  0.12804301
 -0.8339562  -0.18817721  0.41183513 -0.05174294 -0.44192585 -0.4005743
  1.4309162   0.17245919  0.8271333  -0.450793   -0.3791142   0.54197055
 -1.4140967   0.93866676  0.3742875  -0.48446885 -0.09689521  0.45127976
  0.21957451 -0.12921894  1.3608891  -0.13401777 -1.3953071   0.97023785
  0.2134368   0.93396866 -1.3237034   1.5204496  -0.5964158   1.0044338
  0.5189626  -0.55617017  0.24614018  1.4820226   0.26035756  0.83024496
 -0.8764248   0.13766396 -0.4291

In [29]:
# Sample data for demonstration
texts = ["this movie is great", "awesome film", "worst movie ever", "I enjoyed it a lot"]

# Corresponding labels for each text
labels = ["positive", "positive", "negative", "positive"]

# Tokenize the texts and train Word2Vec model
tokenized_texts = [text.split() for text in texts]
word2vec_model = Word2Vec(tokenized_texts, vector_size=100, window=5, min_count=1, workers=4)

# Convert texts to sequences of word indices
word_index = {word: index + 1 for index, word in enumerate(word2vec_model.wv.index_to_key)}
X = [[word_index[word] for word in seq] for seq in tokenized_texts]
X = pad_sequences(X, maxlen=max(len(seq) for seq in tokenized_texts), padding='post')

In [30]:
# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1, output_dim=100, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

/Users/sofarooq/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [32]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Epoch 1/10


AttributeError: module 'keras.src.backend' has no attribute 'convert_to_numpy'

In [23]:
# Sample input text for testing
input_text = "I loved this movie!"

tokenized_input = input_text.split()

# Convert the tokenized input text to word indices, handling out-of-vocabulary words
input_indices = [word_index[word] if word in word_index else 0 for word in tokenized_input]

# Pad the input sequence to match the maximum sequence length
padded_input = pad_sequences([input_indices], maxlen=X.shape[1], padding='post')

# Make predictions using the trained model
prediction = model.predict(padded_input)
predicted_label = "positive" if prediction[0][0] > 0.5 else "negative"

print(f"Input Text: {input_text}")
print(f"Predicted Label: {predicted_label}")

Building LSTM model...
Training LSTM model...
Epoch 1/10


AttributeError: module 'keras.src.backend' has no attribute 'convert_to_numpy'